# Домашняя работа

В этой работы мы будем учиться предсказывать зарплату data scientist-ов в зависимочти от ряда факторов с помощью градиентоного бустинга

Цель домашней работы научиться предсказывать зарплаты (salary_in_usd) по ряды факторов

work_year: The number of years of work experience in the field of data science.

experience_level: The level of experience, such as Junior, Senior, or Lead.

employment_type: The type of employment, such as Full-time or Contract.

job_title: The specific job title or role, such as Data Analyst or Data Scientist.

salary: The salary amount for the given job.

salary_currency: The currency in which the salary is denoted.

salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

employee_residence: The country or region where the employee resides.

remote_ratio: The percentage of remote work offered in the job.

company_location: The location of the company or organization.

company_size: The company’s size is categorized as Small, Medium, or Large.

In [48]:
import pandas as pd
import numpy as np

In [49]:
df = pd.read_csv("ds_salaries.csv")

In [50]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла)

Разделите выборку на train, val, test (80%, 10%, 10%) в качестве таргета выберите salary_in_usd, удалите из признаков (salary) чтобы избежать лика в данных.

In [51]:
df = pd.get_dummies(df, drop_first= True)

In [52]:
from sklearn.model_selection import train_test_split

X = df.drop(['salary_in_usd', 'salary'],axis = 1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=228)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

In [53]:
X_train.shape, X_test.shape, X_val.shape

((2628, 269), (563, 269), (564, 269))

In [54]:
X_train = np.array(X_train)
y_train = np.array(y_train)

## Задание 2 (0.5 балла)

Обучите модель линейной регресии и оцените её качество через mape и rmse

In [55]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

linreg = LinearRegression()
linreg.fit(X_train, y_train)

print('MAPE:',mean_absolute_percentage_error(y_val, linreg.predict(np.array(X_val))))
print('MSE:', np.sqrt(mean_squared_error(y_val, linreg.predict(np.array(X_val)))))

MAPE: 0.4421670476813525
MSE: 51972.35883767933


## Задание 3 (0.5 балла)

Обучите модель бустинга выберите любую из трех библиотек catboost, xgboost, lightgbm и оцените её качество через mape и rmse

In [56]:
!pip install catboost;

In [57]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [58]:
cat.fit(X_train,y_train, verbose = False)

In [60]:
print('MAPE:', mean_absolute_percentage_error(y_val, cat.predict(X_val)))
print('MSE:', np.sqrt(mean_squared_error(y_val, cat.predict(X_val))))

MAPE: 0.4039858476909781
MSE: 49846.52187501451


## Задание 4 (0.5 балла)

Далее предобработайте категориальные признаки с помощью onehot кодирования, labelencoder и счетчиков (mean target encoding) сравните качество моделей с помощью mape и rmse. Необходимо обучить две модели линейную и бустинг. Какие выводы можно сделать? Выберите лучшую из моделей и дальше будем работать с ней

In [61]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [67]:
df = pd.read_csv("ds_salaries.csv")

X = df.drop(['salary_in_usd', 'salary'],axis = 1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=228)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

In [68]:
oh_enc = OneHotEncoder(handle_unknown='ignore')
X_train_oh = oh_enc.fit_transform(X_train)
X_val_oh = oh_enc.transform(X_val)


linreg_oh = LinearRegression()
linreg_oh.fit(X_train_oh, y_train)

print('MAPE for linreg:',mean_absolute_percentage_error(y_val, linreg_oh.predict(X_val_oh)))
print('MSE for linreg:', np.sqrt(mean_squared_error(y_val, linreg_oh.predict(X_val_oh))))


cat_oh = CatBoostRegressor()
cat_oh.fit(X_train_oh,y_train, verbose = False)

print()
print('MAPE for boosting:',mean_absolute_percentage_error(y_val, cat_oh.predict(X_val_oh)))
print('MSE for boosting:', np.sqrt(mean_squared_error(y_val, cat_oh.predict(X_val_oh))))

MAPE for linreg: 0.4430596384298492
MSE for linreg: 51787.75930397386

MAPE for boosting: 0.4036771661004771
MSE for boosting: 50140.56189474383


In [66]:
label_encoder = LabelEncoder()
X_train_le = X_train.copy()
X_val_le = X_val.copy()

for col in X_train.columns.to_list():
    labels = X_train_le[col].unique().tolist()+['unknown']
    label_encoder.fit(labels)
    X_val_le[col] = X_val_le[col].map(lambda s:'unknown' if s not in label_encoder.classes_ else s)
    X_train_le[col] = label_encoder.transform(X_train_le[col])
    X_val_le[col] = label_encoder.transform(X_val_le[col])



linreg_label = LinearRegression()
linreg_label.fit(X_train_le, y_train)

print('MAPE for linreg:',mean_absolute_percentage_error(y_val, linreg_label.predict(X_val_le)))
print('MSE for linreg:', np.sqrt(mean_squared_error(y_val, linreg_label.predict(X_val_le))))

cat_label = CatBoostRegressor()
cat_label.fit(X_train_le,y_train, verbose = False)

print()
print('MAPE for boosting:',mean_absolute_percentage_error(y_val, cat_label.predict(X_val_le)))
print('MSE for boosting:', np.sqrt(mean_squared_error(y_val, cat_label.predict(X_val_le))))

<ipython-input-66-3541326617cc>:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  X_val_le[col] = X_val_le[col].map(lambda s:'unknown' if s not in label_encoder.classes_ else s)
<ipython-input-66-3541326617cc>:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  X_val_le[col] = X_val_le[col].map(lambda s:'unknown' if s not in label_encoder.classes_ else s)


MAPE for linreg: 0.6514078991842985
MSE for linreg: 58399.168505486996

MAPE for boosting: 0.5016971131445227
MSE for boosting: 53525.068660399804


In [71]:
X_train_le.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
1661,3,2,2,32,16,65,0,64,1
3414,2,3,2,45,16,65,1,64,1
1829,2,3,2,40,16,65,0,64,1
2233,2,3,2,30,16,65,1,64,1
2380,2,0,2,66,16,36,1,19,2


In [72]:
X_train_target = X_train_le.copy()
X_val_target = X_val_le.copy()
X_train_target['salary_in_usd'] = y_train

for col in X_train.columns.tolist():
  mean_target = X_train_target.groupby(col)['salary_in_usd'].mean()
  X_train_target[col] = X_train_target[col].map(mean_target)

X_train_target = X_train_target.drop(['salary_in_usd'], axis = 1)

linreg_target = LinearRegression()
linreg_target.fit(X_train_target, y_train)

print('MAPE for linreg:',mean_absolute_percentage_error(y_val, linreg_target.predict(X_val_target)))
print('MSE for linreg:', np.sqrt(mean_squared_error(y_val, linreg_target.predict(X_val_target))))

cat_target = CatBoostRegressor()
cat_target.fit(X_train_target,y_train, verbose = False)

print()
print('MAPE for boosting:',mean_absolute_percentage_error(y_val, cat_target.predict(X_val_target)))
print('MSE for boosting:', np.sqrt(mean_squared_error(y_val, cat_target.predict(X_val_target))))

MAPE for linreg: 3.0223538535164503
MSE for linreg: 326074.81630707794

MAPE for boosting: 0.8142837028564313
MSE for boosting: 133225.84292542015


## Задание 5 (0.5 балла)

Покажите, где ошибается ваша модель. Выведите топ 20 примеров с наибольшей ошибкой. Проанализируете их, постарайтесь выделить группы с причинами ошибок в моделе. Какие выводы можно сделать? Что стоит изменить в данных чтобы улучшить качество модели?

In [73]:
df = pd.read_csv("ds_salaries.csv")
df = pd.get_dummies(df, drop_first= True )

X = df.drop(['salary_in_usd', 'salary'],axis = 1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=228)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

X_train = np.array(X_train)
y_train = np.array(y_train)

linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

In [74]:
errors = [[abs(np.array(y_val)[i] - linreg.predict(np.array([X_val.iloc[1]]))), i] for i in range(X_val.shape[0])]

In [75]:
errors.sort()

In [76]:
list_of_i = []
for i in range(20):
  list_of_i.append(errors[-20:][i][1])

new_list_of_i = X_val.reset_index()['index'].loc[list_of_i].tolist()

In [77]:
df = pd.read_csv("ds_salaries.csv")
X = df.drop(['salary_in_usd', 'salary'],axis = 1)

In [78]:
# Худший предикт
X.loc[new_list_of_i]

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
1558,2023,SE,FT,Data Science Manager,USD,US,0,US,M
2670,2022,SE,FT,Data Engineer,USD,US,0,US,M
2856,2022,SE,FT,Data Engineer,USD,US,0,US,M
3703,2021,MI,FT,Data Scientist,BRL,BR,0,BR,S
1547,2023,EN,FT,Business Data Analyst,EUR,GR,50,GR,L
1341,2023,EN,FT,Data Scientist,INR,IN,50,IN,L
2427,2022,SE,FT,Cloud Data Engineer,EUR,SK,100,SK,S
2034,2022,EN,FT,Machine Learning Engineer,USD,AR,100,AR,L
3735,2021,EN,PT,AI Scientist,USD,BR,100,US,S
358,2023,SE,FT,Machine Learning Engineer,USD,US,100,US,M


In [79]:
# Лучший предикт
list_of_i = []
for i in range(20):
  list_of_i.append(errors[:20][i][1])

new_list_of_i = X_val.reset_index()['index'].loc[list_of_i].tolist()

X.loc[new_list_of_i]

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
3305,2022,SE,FT,Data Engineer,USD,US,100,US,M
2409,2022,SE,FT,Data Scientist,USD,US,100,US,M
1359,2023,SE,FT,Data Scientist,USD,US,100,US,M
1283,2023,SE,FT,Data Science Lead,USD,US,0,US,M
1544,2023,SE,FT,Data Scientist,USD,US,0,US,M
1385,2023,SE,FT,Data Scientist,USD,US,0,US,M
2248,2022,SE,FT,Data Engineer,USD,US,0,US,M
1469,2023,MI,FT,Data Analyst,USD,US,0,US,M
2801,2022,SE,FT,Data Engineer,USD,US,100,US,M
1218,2023,SE,FT,Data Architect,USD,US,100,US,M


Видимо, модель ошибается в случае, если компания не находится в US. В наилучших предсказываниях все люди работали в Омэрике, когда как в худших предсказываниях лишь 9 были из US

## Задание 6 (0.5 балла)

Придумайте признаки для улучшения качества модели на основе предыдущего пункта. Как вам кажется какими признаками можно улучшить качетсво модели? Реализуйте признаки и проверьте улучшилось ли качество модели. Полный бал за задание ставится даже при отсутствии улучшения качества модели, важно попробовать проверить свои гипотезы и сделать выводы

In [80]:
df['in_US'] = (df['company_location'] == 'US').map(lambda x: 1 if x else 0)

df['in_US'] = df['in_US'] * df['remote_ratio'] / 100

In [81]:
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,in_US
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L,0.0
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S,1.0
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S,1.0
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M,0.0
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L,1.0
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L,1.0
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S,1.0
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L,1.0


In [82]:
df = pd.get_dummies(df, drop_first= True )

X = df.drop(['salary_in_usd', 'salary'],axis = 1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=228)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

linreg = LinearRegression()
linreg.fit(X_train, y_train)

print('mean absolute percentage error:',mean_absolute_percentage_error(y_val, linreg.predict(X_val)))
print('mean squared error:', np.sqrt(mean_squared_error(y_val, linreg.predict(X_val))))

mean absolute percentage error: 0.4415562459272788
mean squared error: 51949.38760654537


## Задание 7 (0.5 балла)

Теперь используете исключительно catboost. Обучить модель. Подберите оптимальные гиперпараметры. Используйте pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметры cat_features. Оцените качество итоговой модели, скорость обучения и скорость предсказания

In [83]:
df = pd.read_csv("ds_salaries.csv")

X = df.drop(['salary_in_usd', 'salary'],axis = 1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=228)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

In [85]:
from catboost import Pool, cv
import matplotlib.pyplot as plt

params = {"iterations": 10,
          "depth": 4,
          "loss_function": "RMSE",
          "verbose": False}
cv_dataset = Pool(data=X_train, label=y_train,
                  cat_features = ['experience_level', 'employment_type', 'job_title', 'salary_currency', 'employee_residence', 'company_location', 'company_size'])
scores = cv(cv_dataset,
            params,
            fold_count=3,
            plot="True")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/3]

bestTest = 119405.5834
bestIteration = 9

Training on fold [1/3]

bestTest = 116130.328
bestIteration = 9

Training on fold [2/3]

bestTest = 119433.9381
bestIteration = 9



In [86]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor(verbose=False)

In [87]:
cat_features = ['experience_level', 'employment_type', 'job_title', 'salary_currency', 'employee_residence', 'company_location', 'company_size']

In [90]:
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 7, 10],
        'l2_leaf_reg': [1, 3, 5, 8],
        }


grid_search_result = cat.grid_search(grid, cv_dataset)
print(grid_search_result['params'])
print(grid_search_result['cv_results'])


bestTest = 48949.93553
bestIteration = 268

0:	loss: 48949.9355345	best: 48949.9355345 (0)	total: 2.71s	remaining: 1m 2s

bestTest = 49014.2322
bestIteration = 115

1:	loss: 49014.2322042	best: 48949.9355345 (0)	total: 5.5s	remaining: 1m

bestTest = 48812.22268
bestIteration = 585

2:	loss: 48812.2226793	best: 48812.2226793 (2)	total: 8.29s	remaining: 58s

bestTest = 48842.5126
bestIteration = 259

3:	loss: 48842.5126005	best: 48812.2226793 (2)	total: 12.2s	remaining: 1m

bestTest = 48935.66073
bestIteration = 697

4:	loss: 48935.6607292	best: 48812.2226793 (2)	total: 15.2s	remaining: 57.9s

bestTest = 48748.20823
bestIteration = 136

5:	loss: 48748.2082267	best: 48748.2082267 (5)	total: 18.1s	remaining: 54.2s

bestTest = 48960.87295
bestIteration = 426

6:	loss: 48960.8729471	best: 48748.2082267 (5)	total: 20.9s	remaining: 50.7s

bestTest = 48789.33557
bestIteration = 196

7:	loss: 48789.3355718	best: 48748.2082267 (5)	total: 24.3s	remaining: 48.6s

bestTest = 48650.50344
bestIterati

## Задание 8 (0.5 балла)

Теперь используете исключительно xgboost. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

In [91]:
!pip install xgboost

In [99]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import sklearn

In [100]:
df = pd.read_csv("ds_salaries.csv")
df = pd.get_dummies(df, drop_first= True )

X = df.drop(['salary_in_usd', 'salary'],axis = 1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=228)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)

In [101]:
model = XGBRegressor()

In [102]:
grid={"learning_rate": (0.03, 0.10),
                        "max_depth": [4, 6, 10],
                        "min_child_weight": [ 1, 3, 5, 7],
      }

In [103]:
grid_search = GridSearchCV(
    estimator=model,
    param_grid= grid,
    scoring = 'neg_mean_squared_error',
    cv = 4,
    n_jobs = -1,
    verbose = 0
)

In [104]:
X_train

,work_year,remote_ratio,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT,job_title_AI Developer,job_title_AI Programmer,...,company_location_SG,company_location_SI,company_location_SK,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN,company_size_M,company_size_S
1661,2023,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3414,2022,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1829,2022,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2233,2022,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2380,2022,100,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,2023,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1785,2023,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1907,2022,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3216,2022,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [105]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=4,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': (0.03, 0.1), 'max_depth': [4, 6, 10],
                         'min_child_weight': [1, 3, 5, 7]},
             scoring='neg_mean_squared_error')

In [106]:
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 1}

In [107]:
model = XGBRegressor(learning_rate = 0.1, max_depth = 6, min_child_weight = 1)
model.fit(X_train, y_train)

print('MAPE:',mean_absolute_percentage_error(y_val, model.predict(np.array(X_val))))
print('MSE:', np.sqrt(mean_squared_error(y_val, model.predict(np.array(X_val)))))

MAPE: 0.4013125338047348
MSE: 49281.91256590011


## Задание 9 (0.5 балла)

Теперь используете исключительно lightgbm. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

In [116]:
pip install lightgbm

In [117]:
import lightgbm
from lightgbm import LGBMRegressor

In [118]:
model = LGBMRegressor()

In [119]:
grid={"learning_rate": (0.03, 0.10),
                        "max_depth": [4, 6, 10],
                        "min_child_weight": [ 1, 3, 5, 7],
      }

In [120]:
grid_search = GridSearchCV(
    estimator=model,
    param_grid= grid,
    scoring = 'neg_mean_squared_error',
    cv = 4,
    n_jobs = -1,
    verbose = 0
)

In [124]:
grid_search.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000781 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 78
[LightGBM] [Info] Number of data points in the train set: 2628, number of used features: 37
[LightGBM] [Info] Start training from score 137599.574962
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

GridSearchCV(cv=4, estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'learning_rate': (0.03, 0.1), 'max_depth': [4, 6, 10],
                         'min_child_weight': [1, 3, 5, 7]},
             scoring='neg_mean_squared_error')

In [125]:
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 1}

In [126]:
model = LGBMRegressor(learning_rate = 0.1, max_depth = 6, min_child_weight = 1)
model.fit(X_train, y_train)

print('MAPE:', mean_absolute_percentage_error(y_val, model.predict(np.array(X_val))))
print('MSE:', np.sqrt(mean_squared_error(y_val, model.predict(np.array(X_val)))))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000853 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 78
[LightGBM] [Info] Number of data points in the train set: 2628, number of used features: 37
[LightGBM] [Info] Start training from score 137599.574962
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

## Задание 10 (0.5 балла)

Сделайте выводы про модели бустинга, какая из моделей показала лучший результат по качеству, скорости обучения и скорости предсказания

In [127]:
%%time
for i in range(10):
  model = LGBMRegressor(learning_rate = 0.1, max_depth = 6, min_child_weight = 1)
  model.fit(X_train, y_train)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000847 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 78
[LightGBM] [Info] Number of data points in the train set: 2628, number of used features: 37
[LightGBM] [Info] Start training from score 137599.574962
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

In [128]:
%%time
print('MAPE:', mean_absolute_percentage_error(y_val, model.predict(np.array(X_val))))
print('MSE:', np.sqrt(mean_squared_error(y_val, model.predict(np.array(X_val)))))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
MAPE: 0.4063914470527593
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
MSE: 49807.38580508824
CPU times: user 23.3 ms, sys: 9 µs, total: 23.3 ms
Wall time: 37.4 ms


In [129]:
%%time
for i in range(10):
  model = XGBRegressor(learning_rate = 0.1, max_depth = 6, min_child_weight = 1)
  model.fit(X_train, y_train)

CPU times: user 1min 11s, sys: 173 ms, total: 1min 12s
Wall time: 57.8 s


In [130]:
%%time
print('MAPE:', mean_absolute_percentage_error(y_val, model.predict(np.array(X_val))))
print('MSE:', np.sqrt(mean_squared_error(y_val, model.predict(np.array(X_val)))))

MAPE: 0.4013125338047348
MSE: 49281.91256590011
CPU times: user 22.9 ms, sys: 0 ns, total: 22.9 ms
Wall time: 13 ms


In [131]:
%%time
for i in range(10):
  model = CatBoostRegressor(verbose=False, depth = 6, l2_leaf_reg = 3, learning_rate = 0.03)
  model.fit(X_train, y_train)

CPU times: user 24.6 s, sys: 3.11 s, total: 27.7 s
Wall time: 16.3 s


In [132]:
print('MAPE:', mean_absolute_percentage_error(y_val, model.predict(np.array(X_val))))
print('MSE:', np.sqrt(mean_squared_error(y_val, model.predict(np.array(X_val)))))

MAPE: 0.4063603308013601
MSE: 49704.2119864477
